In [68]:
def addSabermetrics(df):
    df["battingSingles"] = (df["battingHits"] - df["battingDoubles"] - df["battingTriples"] - df["battingHomeRuns"])
    df["totalBases"] = (df["battingSingles"] + 2*df["battingDoubles"] + 3*df["battingTriples"] + 4*df["battingHomeRuns"])
    df["battingSlg"] = (df["totalBases"]/df["battingAtBats"])
    df["OBP+Slg"] = (df["onBasePercentage"] + df["battingSlg"])
    df["pitchingSingles"] = (df["pitchingHits"] - df["pitchingDoubles"] - df["pitchingTriples"] - df["pitchingHomeRuns"])
    df["pitchingRunsAllowedPerInning"] = (df["pitchingEarnedRuns"]/df["pitchingInningsPitched"])
    df["pitchingWinPercentage"] = (df["pitchingWins"] / (df["pitchingWins"] + df["pitchingLosses"]))
    df["pitchingWalks+Hits"] = (df["pitchingHits"] + df["pitchingBaseOnBalls"])
    df["pitchingWalks+HitsPercentage"] = ((df["pitchingHits"] + df["pitchingBaseOnBalls"]) / df["pitchingAtBats"])
    df["pitchingWalks+HitsPerInning"] = ((df["pitchingHits"] + df["pitchingBaseOnBalls"]) / df["pitchingInningsPitched"])
    df.fillna(0, inplace=True)
    

In [69]:
%pip install --upgrade pip
%pip install pybaseball
%pip install pandas
%pip install scikit-learn
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np



ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\python312\\lib\\site-packages\\pip-23.2.1.dist-info\\AUTHORS.txt'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
START = 2005
END = 2025

In [71]:
from pybaseball import batting_stats
from pybaseball import pitching_stats
from pybaseball import fielding_stats
batting = batting_stats(START,END,qual=200)
pitching = pitching_stats(START,END,qual=50)
fielding = fielding_stats(START,END)

In [72]:
batting.to_csv("batting.csv")
pitching.to_csv("batting.csv")
fielding.to_csv("batting.csv")
csvs = [batting,pitching,fielding]

In [73]:
for i in csvs:
    i = i.groupby("IDfg",group_keys=False).filter(lambda x: x.shape[0] > 1)

In [74]:
def next_season(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    return player


In [75]:
batting = batting.groupby("IDfg",group_keys=False).apply(next_season)
pitching = pitching.groupby("IDfg",group_keys=False).apply(next_season)

C:\Users\riley\AppData\Local\Temp\ipykernel_24652\2768861365.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting = batting.groupby("IDfg",group_keys=False).apply(next_season)
C:\Users\riley\AppData\Local\Temp\ipykernel_24652\2768861365.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pitching = pitching.groupby("IDfg",group_keys=False).apply(next_season)


In [76]:
batting[["Name","Season","WAR","Next_WAR"]]

,Name,Season,WAR,Next_WAR
4988,Alfredo Amezaga,2006,1.1,2.0
4468,Alfredo Amezaga,2007,2.0,1.2
4690,Alfredo Amezaga,2008,1.2,NaN
3696,Garret Anderson,2005,-0.2,0.1
3485,Garret Anderson,2006,0.1,1.4
...,...,...,...,...
2466,Seiya Suzuki,2022,2.1,3.2
1185,Seiya Suzuki,2023,3.2,NaN
6315,Oscar Colas,2023,-1.3,NaN
4620,Zach Neto,2023,1.1,NaN


In [77]:
nulls = batting.isnull().sum()

In [78]:
noNullColumns = list(batting.columns[nulls == 0])
batting = batting[noNullColumns + ["Next_WAR"]].copy()

In [79]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR
4988,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,107,113,143,109,63,0,0.188,0.256,1.1,2.0
4468,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,101,112,109,113,75,0,0.175,0.227,2.0,1.2
4690,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,101,101,123,111,64,0,0.178,0.244,1.2,NaN
3696,2,2005,Garret Anderson,LAA,33,142,575,603,163,111,...,78,83,100,97,106,0,0.150,0.258,-0.2,0.1
3485,2,2006,Garret Anderson,LAA,34,141,543,588,152,105,...,96,86,88,94,119,0,0.155,0.253,0.1,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,30116,2022,Seiya Suzuki,CHC,27,111,397,446,104,66,...,105,130,106,95,106,290,0.232,0.312,2.0,3.2
1185,30116,2023,Seiya Suzuki,CHC,28,138,515,583,147,90,...,109,92,90,93,115,392,0.204,0.283,2.9,NaN
6315,30162,2023,Oscar Colas,CHW,24,75,245,263,53,39,...,109,132,142,89,99,178,0.144,0.307,-1.6,NaN
4620,31347,2023,Zach Neto,LAA,22,84,289,329,65,39,...,82,103,81,108,97,216,0.161,0.290,0.8,NaN


In [80]:
batting.dtypes[batting.dtypes == "object"]

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [81]:
del batting["Dol"]


In [82]:
del batting["Age Rng"]

In [83]:
batting.dtypes[batting.dtypes == "object"]

Name    object
Team    object
dtype: object

In [84]:
batting["team_codes"] = batting["Team"].astype("category").cat.codes

In [85]:
batting_full = batting.copy()
batting = batting.dropna()

In [86]:
pitching[["Name","Season","WAR","Next_WAR"]]

,Name,Season,WAR,Next_WAR
830,Neftali Feliz,2010,2.0,1.0
852,Neftali Feliz,2011,1.0,-0.1
2171,Neftali Feliz,2016,-0.1,NaN
5303,Ramon Ortiz,2005,0.3,0.2
5486,Ramon Ortiz,2006,0.2,0.1
...,...,...,...,...
1708,Gavin Williams,2023,1.1,NaN
1176,Tanner Bibee,2023,3.0,NaN
3638,Bryan Woo,2023,1.0,NaN
1173,Kodai Senga,2023,3.4,NaN


In [87]:
nulls = pitching.isnull().sum()

In [88]:
noNullColumns = list(pitching.columns[nulls == 0])
pitching = pitching[noNullColumns + ["Next_WAR"]].copy()

In [89]:
pitching

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,Next_WAR
830,18,2010,Neftali Feliz,TEX,22,4,3,2.0,2.73,70,...,78,88,151,134,99,80,0,0.165,0.284,1.0
852,18,2011,Neftali Feliz,TEX,23,2,3,1.0,2.74,64,...,91,106,106,106,89,119,0,0.159,0.274,-0.1
2171,18,2016,Neftali Feliz,PIT,28,4,2,-0.1,3.52,62,...,92,78,142,102,88,118,135,0.117,0.259,NaN
5303,27,2005,Ramon Ortiz,CIN,32,9,11,0.3,5.36,30,...,110,103,80,80,104,104,0,0.147,0.244,0.2
5486,27,2006,Ramon Ortiz,WSN,33,11,16,0.2,5.57,33,...,100,106,91,97,102,96,0,0.148,0.231,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708,30122,2023,Gavin Williams,CLE,23,3,5,1.1,3.29,16,...,86,111,108,81,115,86,223,0.157,0.278,NaN
1176,30134,2023,Tanner Bibee,CLE,24,10,4,3.0,2.98,25,...,93,108,99,108,102,93,394,0.165,0.276,NaN
3638,30279,2023,Bryan Woo,SEA,23,4,5,1.0,4.21,18,...,88,99,121,113,107,83,240,0.152,0.276,NaN
1173,31838,2023,Kodai Senga,NYM,30,12,7,3.4,2.98,29,...,101,96,104,104,98,101,410,0.155,0.280,NaN


In [90]:
pitching.dtypes[pitching.dtypes == "object"]

Name       object
Team       object
Dollars    object
Age Rng    object
dtype: object

In [91]:
del pitching["Dollars"]

In [92]:
del pitching["Age Rng"]

In [93]:
pitching["team_codes"] = pitching["Team"].astype("category").cat.codes

In [94]:
pitching_full = pitching.copy()
pitching = pitching.dropna()